In [ ]:
!pip install pydantic

In [ ]:
!pip install --upgrade boto3

In [ ]:
!pip install --upgrade langchain

# This notebook is used to evaluate LLAMA2 for RAG with Amazon Kendra . This notebook uses custom Langchain Agents with Amazon Kendra as a Tool for RAG

## Full fledged Chatbot (Agents approch) implementation are avaialbe at :
### Financial Bot : https://demos.aiml.aws.dev/intro.html?demo=conv-search-finance
### Clinical Trail Bot :https://demos.aiml.aws.dev/intro.html?demo=clinical-trials-bot
##### **Above 2 demos use Anthropic model
##### **Reach out to Abhishek Maligehalli Shivalingaiah (@abhmalig) if you have questions


# Based on experimentation in this notebook LLAMA2 13B/70B LLM seems like a *VIABLE* choice to build such demos (chatbots)

In [26]:
from langchain import SagemakerEndpoint #,Kendra
from langchain.docstore.document import Document
from langchain.agents import load_tools,initialize_agent,Tool
from langchain.agents.react.base import DocstoreExplorer
from langchain.llms.sagemaker_endpoint import ContentHandlerBase,LLMContentHandler
from typing import Dict ,List
import json
#import Anthropic
from langchain.llms import Anthropic
from langchain.prompts import StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser 

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.docstore.base import Docstore
import re



## Kendra as Docstore

In [27]:
class Kendra(Docstore):
    """Wrapper around Kendra API."""

    def __init__(self,kendra_index_id :str, region_name:str) -> None:
        """Check that boto3 package is installed."""
        
        self.used = False
        self.URL = ""
        
        try:
            import boto3
            self.kendra_client = boto3.client("kendra",region_name=region_name)
            self.s3_client = boto3.client("s3")
            self.kendra_index_id = kendra_index_id
            
        except ImportError:
            raise ValueError(
                "Could not import boto3 python package. "
                "Please it install it with `pip install boto3`."
            )

    def parseResponse(self,response):
        for each_loop in response['ResultItems'][0]['DocumentAttributes']:
            if (each_loop['Key']=='_excerpt_page_number'):
                pagenumber = each_loop['Value']['LongValue'] -1   
        return pagenumber
    
    def parseBucketandKey(self,SourceURI):
        return (SourceURI.split('/', 3)[2],SourceURI.split('/', 3)[3])


    #def search(self, query : str ) -> str, Document]:
    def search(self, query : str ) -> str:
        """Try to search for a document in Kendra Index""
        
        """
        try:
            page_size = 4
            page_number = 1

            result =  self.kendra_client.retrieve(
                    IndexId = self.kendra_index_id,
                    QueryText = query,
                    PageSize = page_size,
                    PageNumber = page_number)
        except:
            return "RELAVENT PASSAGES NOT FOUND"

        # Concatinating the results from Kendra Retreive API 
        # https://docs.aws.amazon.com/kendra/latest/dg/searching-retrieve.html
        context =""
        for retrieve_result in result["ResultItems"]:
            context =context +'['
            context = context + "Title: " + str(retrieve_result["DocumentTitle"] + ", URI: " + str(retrieve_result["DocumentURI"]) +", Passage content: " + str(retrieve_result["Content"]))
            context =context + '] '
        return context

In [28]:
kendra_docstore = Kendra(kendra_index_id ="ea70f4fe-e80b-432b-b160-8cb33205930f",region_name='us-east-1')
#kendra_docstore.search("what is the agreement signing date EcoScience Solutions contract ? ")
#kendra_docstore.search("EcoScience Solutions contract date ? ")
#kendra_docstore.search("EcoScience Solutions contract  ? ")
kendra_docstore.search("what is the effective date for")

'[Title: Antares Pharma, Inc. - Manufacturing Agreement, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/legal/Manufacturing/Antares Pharma, Inc. - Manufacturing Agreement.PDF, Passage content: ALL SUCH OMITTED MATERIAL HAS BEEN FILED WITH THE SECURITIES AND EXCHANGE COMMISSION PURSUANT TO RULE 24b-2 PROMULGATED UNDER THE SECURITIES EXCHANGE ACT OF 1934, AS AMENDED “Binding Forecast” has the meaning specified in Section 3.2(a); “[***]” has the meaning specified in Section 3.2(c); “Business Day” means a day other than a Saturday, Sunday or a day that is a federal holiday in the United States; “Calendar Quarter” means a three-month period ending on March 31, June 30, September 30 or December 31; , Sunday or a day that is a federal holiday in the United States; “Calendar Year” means a calendar year occurring after the Effective Date; provided, however, the first Calendar Year means the period from the Effective Date up to and including December 31 of the same calendar year in which t

In [29]:
kendra_docstore = Kendra(kendra_index_id ="ea70f4fe-e80b-432b-b160-8cb33205930f",region_name='us-east-1')
#kendra_docstore.search("what is the agreement signing date EcoScience Solutions contract ? ")
#kendra_docstore.search("EcoScience Solutions contract date ? ")
#kendra_docstore.search("EcoScience Solutions contract  ? ")
kendra_docstore.search("how much time off  is available for adoptive assistance?")

'[Title: PaidFamilyLeaveProgram_California, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/HR/PaidFamilyLeaveProgram_California.pdf, Passage content: Caregiving: Available to customers to care for a seriously ill child, parent, parent-in-law, grandparent, grandchild, sibling, spouse, or registered domestic partner. PFL defines “seriously ill” as an illness, injury, impairment, or physical or mental condition that requires: • At-home care or in-patient care in a hospital, hospice, or residential medical facility. • Continuing treatment by a licensed health professional. Bonding: Available to parents welcoming a new child into the family through birth, adoption, or foster care placement within the past 12 months. Foster and adopted children must be under the age of 18. A “parent” is defined as an individual, spouse, or registered domestic partner who will be serving as a parent for the child. “Paid Family Leave provided the peace of mind when it provided benefits to me, so I can fo

In [30]:
kendra_docstore.search("what is the amount paid to Fruit of Life Productions LLC? ")

'[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/legal/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformation Information See Attached Wire Instructions Sponsorship Benefits for Presenting Partner Sponsor: * Main Stage named after your brand * 4 10x10 on site vendor booths *

## LLAMA2 13B/70B chat JSON handler

In [33]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        dialog = [{"role": "user", "content": prompt}]
        input_str = json.dumps({ "inputs" : [dialog], **model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        model_predictions = json.loads(output.read())
        return model_predictions[0]['generation']['content']
content_handler = ContentHandler()

# Custom AGENT LLM

In [34]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor ,ReActChain
from langchain import  LLMChain

## Defining Kendra as a TOOL for Langchain Agents

In [35]:
kendra_docstore = Kendra(kendra_index_id ="ea70f4fe-e80b-432b-b160-8cb33205930f",region_name='us-east-1')

tools = [
    Tool(
        name="Search",
        func=kendra_docstore.search,
        description="useful for when you need to answer legal questions regarding contract ,regulations, policies"
    )
]

In [331]:
#endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-70b-f",
llm = SagemakerEndpoint(
            endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-70b-f",
            region_name="us-east-1",
            content_handler=content_handler,
            endpoint_kwargs={'CustomAttributes':'accept_eula=true'}
    )

## Custom Prompt template and parser 

In [332]:
# Set up the base template
template = """You are a conversational AI bot, Answer the following questions as best you can in less sentences as possible. 
You have access to the following tools:

{tools}

To use a tool, STRICTLY follow below format:

```
Thought: Do I need to use a tool? YES
Action: the action to take, should be one of [{tool_names}]
Action Input: the search key words ONLY
Observation: the result of the search
```
YOU MUST USE THE BELOW FORMATE ,if you have a response to say to the user, or if you do not need to use a tool, 
YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

```
Thought: Do I need to use a tool? NO
AI: [your response here]


Example 1 Start:
Question: What is the  population of India?
Thought: Do I need to use a tool? YES
Action: Search
Action Input: population of India
Observation:  India is in Asia.The current estimated population of India is approximately 1.38 billion. Its capital is D
Thought: Do I need to use a tool again ? NO
AI: The population of India is approximately 1.38 billion based on the search results
Example 1 End

Example 2 Start:
Question: How are you?
Thought: Do I need to use a tool? NO
AI: I'm doing good. How are you?
Example 2 End

YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

Begin!


New input: {input}
{agent_scratchpad}"""

In [333]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)


In [334]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input","intermediate_steps"]
)

In [335]:
print(prompt.template)

You are a conversational AI bot, Answer the following questions as best you can in less sentences as possible. 
You have access to the following tools:

{tools}

To use a tool, STRICTLY follow below format:

```
Thought: Do I need to use a tool? YES
Action: the action to take, should be one of [{tool_names}]
Action Input: the search key words ONLY
Observation: the result of the search
```
YOU MUST USE THE BELOW FORMATE ,if you have a response to say to the user, or if you do not need to use a tool, 
YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

```
Thought: Do I need to use a tool? NO
AI: [your response here]


Example 1 Start:
Question: What is the  population of India?
Thought: Do I need to use a tool? YES
Action: Search
Action Input: population of India
Observation:  India is in Asia.The current estimated population of 

In [336]:
class CustomOutputParser(AgentOutputParser):
    ai_prefix: str = "AI"
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        if f"{self.ai_prefix}:" in text:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        regex = r"Action: (.*?)[\n]*Action Input: ((.|\n)*)"
        match = re.search(regex, text)
        if not match:
            
            print("text :",text)
            raise OutputParserException(f"Could not parse LLM output: `{text}`")
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

In [337]:
output_parser = CustomOutputParser()
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [338]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [339]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Evaluation of LLAMA2(13/70b) to power (Langchain)Agents and Tool usage

## 1. LLAMA2 decided to NOT to use tool(Kendra) for casual conversation

In [340]:
agent_executor.run("I'm Abhishek ,who are you")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9976fa0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...


Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e997f370>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Thought: Do I need to use a tool? NO
AI: Hello Abhishek, I'm just an AI, I don't have a personal identity, but I'm here to help you with any questions or information you might need. How can I assist you today?

> Finished chain.


"Hello Abhishek, I'm just an AI, I don't have a personal identity, but I'm here to help you with any questions or information you might need. How can I assist you today?"

In [341]:
agent_executor.run("how are you today ")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e99e3fd0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...


Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9976a90>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Thought: Do I need to use a tool? NO
AI: I'm doing well, thanks for asking! How about you?

> Finished chain.


"I'm doing well, thanks for asking! How about you?"

## 2. LLAMA2 decided to use tool(Kendra) for searching about contracts

In [342]:
try:
    agent_executor.run("what is the effective date of EcoScience Solutions contract?")
except ValueError as e:
    response = str(e)
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e99874f0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: EcoScience Solutions contract effective date

Observation:[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformat

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9987d60>: Failed to establish a new connection: [Errno 111] Connection refused'))


 AI: The effective date of EcoScience Solutions contract is April 1st, 2018.

> Finished chain.


In [343]:
agent_executor.run("how long fulltime employees have to work")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9976100>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: full-time employees work hours

Observation:[Title: New York State Workers’ Compensation Law, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/HR/New York State Workers’ Compensation Law.pdf, Passage content: b. For any day of family leave during which the employee performed work for the employer for remuneration or profit. c. For any family leave commencing before the employee becomes eligible for PFL benefits. d. For any disability of the employee. B. Eligibility 1. Full-time employees who work a regular schedule of 20 hours or more per week are eligible for Paid Family Leave after 26 consecutive weeks of employment, commencing with the employee’s City Start date. 2. Part-time employees who work a regular schedule of less than 20 hours per week are eligible after working 175 days, which need not be consecutive, commencing with the employee’s City Start date. 3. The annual p

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e997f610>: Failed to establish a new connection: [Errno 111] Connection refused'))


 AI: Based on the search results, it appears that full-time employees in New York State are entitled to Paid Family Leave (PFL) after working 26 consecutive weeks of employment, commencing with their City Start date. Part-time employees who work a regular schedule of less than 20 hours per week are eligible after working 175 days, which need not be consecutive, commencing with their City Start date. The annual period during which an employee is eligible for a defined number of weeks of PFL is defined as a rolling 12-month period measured backward from the first date that the employee begins using PFL. Additionally, full-time employees who work a regular schedule of 20 hours or more per week are eligible for PFL after 26 consecutive weeks of employment, commencing with their City Start date.

It's also worth noting that the Consumer Price Index (CPI) is a measure of inflation and is used to adjust the FTE Rate on an annual basis. The FTE Rate includes certain allowable allocations and i

"Based on the search results, it appears that full-time employees in New York State are entitled to Paid Family Leave (PFL) after working 26 consecutive weeks of employment, commencing with their City Start date. Part-time employees who work a regular schedule of less than 20 hours per week are eligible after working 175 days, which need not be consecutive, commencing with their City Start date. The annual period during which an employee is eligible for a defined number of weeks of PFL is defined as a rolling 12-month period measured backward from the first date that the employee begins using PFL. Additionally, full-time employees who work a regular schedule of 20 hours or more per week are eligible for PFL after 26 consecutive weeks of employment, commencing with their City Start date.\n\nIt's also worth noting that the Consumer Price Index (CPI) is a measure of inflation and is used to adjust the FTE Rate on an annual basis. The FTE Rate includes certain allowable allocations and is 

In [344]:
agent_executor.run("what is the effective date of EcoScience Solutions contract?")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e998d760>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: EcoScience Solutions contract effective date

Observation:[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformat

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e997f4c0>: Failed to establish a new connection: [Errno 111] Connection refused'))


 AI: The effective date of EcoScience Solutions contract is April 1st, 2018.

> Finished chain.


'The effective date of EcoScience Solutions contract is April 1st, 2018.'

In [57]:
agent_executor.run("when will ESSI company have right to terminate Talent ?")



> Entering new AgentExecutor chain...
 Sure, I'll do my best to assist you!

Thought: Do I need to use a tool? YES

Action: Search

Action Input: when will ESSI company have right to terminate Talent


Observation:[Title: EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Endorsement/EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement.pdf, Passage content: Such termination shall relieve ESSI of its obligation to provide any further consideration pursuant to this Agreement. B. Talent shall have the right to terminate this Agreement upon ten (10) days prior written notice to ESSI in the event of the occurrence of any of the following: (1) ESSI adjudicates as insolvent or declares bankruptcy; or (2) ESSI fails to provide consideration due pursuant to this Agreement, within ten (10) days following the date such consideration is due hereunder, provided that ESSI i

'Based on the provided information, it appears that the termination of the agreement can be done by either party with a 10-day prior written notice. Additionally, the agreement can be terminated by ESSI if Talent breaches any of the provisions, including failing to perform the duties set forth in Paragraph 4 or breaching any other covenant or agreement set forth in the agreement. If the agreement is terminated, ESSI shall not be obligated to provide any further consideration pursuant to the agreement.'

In [557]:
agent_executor.run("summarize leaves available for adoptive assistance based on policy?")



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: "adoptive assistance policy"

Observation:[Title: New York State Workers’ Compensation Law, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/HR/New York State Workers’ Compensation Law.pdf, Passage content: 440-16 Subject: Paid Family Leave Benefits for Represented Employees Source: New York State Workers’ Compensation Law, Article 9 Supersedes: 440-16 issued December 21, 2021 Date:   December 19, 2022 1. INTRODUCTION Employees in titles represented by unions that have collectively bargained an agreement to opt into New York State Paid Family Leave Benefits will be eligible to take paid family leave (“PFL”). PFL is an employee-funded insurance policy that provides job-protected, paid time-off to bond with a newly born, adopted, or foster child, to care for a family member with a serious health condition, including preparation and recovery from surgery related to organ or tiss

'Do you have any other questions or concerns regarding adoptive assistance policies?'

In [345]:
agent_executor.run("who is Michael Gorzynski?")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e997fd60>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: Michael Gorzynski

Observation:[Title: HC2HOLDINGS,INC_05_14_2020-EX-10.1-COOPERATION AGREEMENT, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Collaboration/HC2HOLDINGS,INC_05_14_2020-EX-10.1-COOPERATION AGREEMENT.PDF, Passage content: “Avie” Glazer is the principal of Lancer Capital. In addition, he currently serves as Executive Co-Chairman and Director of Manchester United Plc (NYSE: MANU). Mr. Glazer served as President and Chief Executive Officer of Zapata Corporation, a U.S. public company between from March 1995 to July 2009 and Chairman of the board of Zapata Corporation from March 2002 to July 2009. In addition to his professional experience, Mr. Glazer received a business degree from Washington University in St. Louis and received a law degree from American University, Washington College of Law. Michael Gorzynski is the Managing Member of MG Capital, an 

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9987070>: Failed to establish a new connection: [Errno 111] Connection refused'))


text :  Based on the information provided, Michael Gorzynski is the Managing Member of MG Capital, an investment firm focused on complex value-oriented investments. He has a background in investing in special situations globally at Third Point LLC, and has expertise in restructurings and in the insurance and banking industries. He is also a director of HC2 Holdings, Inc. and has been involved in the company's collaboration with stockholders to refresh the board of directors.


NameError: name 'OutputParserException' is not defined

In [487]:
agent_executor.run("who pays for the retouch of images of photo session according to contract?")



> Entering new AgentExecutor chain...
 Sure, I'd be happy to help!

Thought: Do I need to use a tool? YES

Action: Search

Action Input: retouch of images photo session contract


Observation:[Title: AMBASSADOREYEWEARGROUPINC_11_17_1997-EX-10.28-ENDORSEMENT AGREEMENT, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/legal/Endorsement Agreement/AMBASSADOREYEWEARGROUPINC_11_17_1997-EX-10.28-ENDORSEMENT AGREEMENT.PDF, Passage content: The photo sessions shall be scheduled at KI's convenience upon not less than two (2) weeks prior notice, and shall be subject to KI's preexisting personal and professional commitments. The photo sessions shall be up to two (2) consecutive days in duration, each day to consist of no more than eight (8) working hours. The photo shoots shall be produced by SW, approved in writing by Diplomat (which approval shall not be unreasonably withheld), and all images produced from the photo sessions shall be the property of SW. The parties agree that all images fr

'Based on the search results, it appears that the retouching of images is the responsibility of Diplomat, and it is included in the sum paid to SW'

In [490]:
agent_executor.run("when will Talent be able to terminate agreement with ESSI? explain")



> Entering new AgentExecutor chain...
 Sure, I'll do my best to assist you with your questions!

Thought: Do I need to use a tool? YES

Action: Search

Action Input: when will Talent be able to terminate agreement with ESSI?


Observation:[Title: EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Endorsement/EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement.pdf, Passage content: Such termination shall relieve ESSI of its obligation to provide any further consideration pursuant to this Agreement. B. Talent shall have the right to terminate this Agreement upon ten (10) days prior written notice to ESSI in the event of the occurrence of any of the following: (1) ESSI adjudicates as insolvent or declares bankruptcy; or (2) ESSI fails to provide consideration due pursuant to this Agreement, within ten (10) days following the date such consideration is due hereun

'I understand that you would like me to assist you with answering a question regarding the termination of an agreement. Please provide the actual agreement document or the relevant passage content, and I will be happy to help you with your question.'

In [555]:
agent_executor.run("what are the holidays for fulltime employees?")



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: employee holidays

Observation:[Title: THERAVANCEBIOPHARMA,INC_05_08_2020-EX-10.2-SERVICE AGREEMENT, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Service/THERAVANCEBIOPHARMA,INC_05_08_2020-EX-10.2-SERVICE AGREEMENT.PDF, Passage content: The Executive’s participation in any such benefit plans are subject to the terms and conditions of any such plan. The Company reserves the right to discontinue, vary or amend each benefit plan (including the level of cover) at any time on reasonable notice to the Executive. 12. Holidays 12.1 The Executive shall be entitled to 28 days' paid holiday in each holiday year (which includes public holidays) to be taken at times convenient to the Company and authorised in advance. 12.2 The Company's holiday year runs from 1 January to 31 December. If the Executive’s employment commences or terminates part way through a holiday year, the 

"Based on the search results, it appears that full-time employees are entitled to 28 days of paid holiday per year, which includes public holidays. The holiday year runs from January 1 to December 31, and employees are required to take their holidays at times convenient to the company and authorized in advance. If an employee's employment commences or terminates partway through a holiday year, their entitlement during that year will be calculated on a pro-rata basis rounded up to the nearest whole day.\n\nIt's also worth noting that employees have no entitlement to payment in lieu of accrued but untaken holiday, except on termination of employment. In such cases, the amount of payment in lieu shall be 1/260 of the employee's salary for each untaken day of entitlement.\n\nFinally, it's important to mention that the company reserves the right to discontinue, vary, or amend each benefit plan, including holidays, at any time on reasonable notice to the employee."

## 3. LLAMA2 figured out to use or not to use Tool(Kendra) depending on the question and it does consistently.

In [346]:
agent_executor.run("what is the amount paid to Fruit of Life Productions LLC?")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9987b80>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: amount paid to Fruit of Life Productions LLC

Observation:[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/legal/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformation 

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9959250>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Based on the information provided, the amount paid to Fruit of Life Productions LLC by Eco Science Solutions, Inc. is $250,000.00. This is stated in the Sponsorship Agreement between the two parties, which was signed on April 1, 2018. The agreement outlines the terms and conditions of the sponsorship, including the amount to be paid, the benefits of sponsorship, and the intellectual property rights of Fruit of Life Productions LLC.

AI: The amount paid to Fruit of Life Productions LLC by Eco Science Solutions, Inc. is $250,000.00, as stated in the Sponsorship Agreement between the two parties.

> Finished chain.


'The amount paid to Fruit of Life Productions LLC by Eco Science Solutions, Inc. is $250,000.00, as stated in the Sponsorship Agreement between the two parties.'

.

.

.

# Switching gears to introduce Memory to Agents..

.

.

.

# Conversation Agent (for Chat Models) with LLAMA2

In [281]:
import os

In [347]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent
#from langchain import Kendra
from langchain.agents import initialize_agent, Tool
from langchain.agents.react.base import DocstoreExplorer
from langchain import SagemakerEndpoint 

In [348]:
kendra_docstore = Kendra(kendra_index_id ="ea70f4fe-e80b-432b-b160-8cb33205930f",region_name='us-east-1')
tools = [
    Tool(
        name="Search",
        func=kendra_docstore.search,
        description="useful for when you need to answer  questions about Legal contracts,Agreements from document store"
    )
]

In [349]:
# Set up the base template
template = """You are a conversational AI bot, Answer the following questions as best you can in less sentences as possible. 
You have access to the following tools:

{tools}

To use a tool, STRICTLY follow below format:

```
Thought: Do I need to use a tool? YES
Action: the action to take, should be one of [{tool_names}]
Action Input: the search key words ONLY
Observation: the result of the search
```
YOU MUST USE THE BELOW FORMATE ,if you have a response to say to the user, or if you do not need to use a tool, 
YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

```
Thought: Do I need to use a tool? NO
AI: [your response here]


Example 1 Start:
Question: What is the  population of India?
Thought: Do I need to use a tool? YES
Action: Search
Action Input: population of India
Observation:  India is in Asia.The current estimated population of India is approximately 1.38 billion. Its capital is D
Thought: Do I need to use a tool again ? NO
AI: The population of India is approximately 1.38 billion based on the search results
Example 1 End

Example 2 Start:
Question: How are you?
Thought: Do I need to use a tool? NO
AI: I'm doing good. How are you?
Example 2 End

YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

Begin!


Previous conversation history:
{history}

New input: {input}
{agent_scratchpad}"""

In [350]:
print(prompt.template)

You are a conversational AI bot, Answer the following questions as best you can in less sentences as possible. 
You have access to the following tools:

{tools}

To use a tool, STRICTLY follow below format:

```
Thought: Do I need to use a tool? YES
Action: the action to take, should be one of [{tool_names}]
Action Input: the search key words ONLY
Observation: the result of the search
```
YOU MUST USE THE BELOW FORMATE ,if you have a response to say to the user, or if you do not need to use a tool, 
YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER ,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER,YOU MUST ALWAYS USE PREFIX "AI:" TO RESPOND TO USER with FINAL ANSWER

```
Thought: Do I need to use a tool? NO
AI: [your response here]


Example 1 Start:
Question: What is the  population of India?
Thought: Do I need to use a tool? YES
Action: Search
Action Input: population of India
Observation:  India is in Asia.The current estimated population of 

In [351]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)


In [352]:
class CustomOutputParser(AgentOutputParser):
    
    ai_prefix: str = "AI"

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        if f"{self.ai_prefix}:" in text:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        regex = r"Action: (.*?)[\n]*Action Input: (.*)"
        match = re.search(regex, text)
        if not match:
            return AgentFinish({"output": text}, text)
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

In [353]:
output_parser = CustomOutputParser()

In [354]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input","intermediate_steps","history"]
)

In [355]:
memory=ConversationBufferWindowMemory(k=4)

In [356]:
tool_names = [tool.name for tool in tools]
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [357]:
agent= LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names,
    verbose=True,
   
)

In [358]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True,memory=memory)

In [359]:
memory.chat_memory.messages

[]

In [360]:
agent_executor.run(input="my name is Abhishek")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e98b6820>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...


Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e98b6f40>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Thought: Do I need to use a tool? NO
AI: Hello Abhishek, it's nice to meet you. How can I assist you today?

> Finished chain.


"Hello Abhishek, it's nice to meet you. How can I assist you today?"

## 4. LLAMA2 recalls from conversation history (Difficulty Level : EASY)

In [361]:
agent_executor.run(input="what is my name")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e98b6340>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...


Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9920e80>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Thought: Do I need to use a tool? NO
AI: Your name is Abhishek, based on our previous conversation. Is there anything else I can help you with?

> Finished chain.


'Your name is Abhishek, based on our previous conversation. Is there anything else I can help you with?'

In [362]:
memory.chat_memory.messages

[HumanMessage(content='my name is Abhishek', additional_kwargs={}, example=False),
 AIMessage(content="Hello Abhishek, it's nice to meet you. How can I assist you today?", additional_kwargs={}, example=False),
 HumanMessage(content='what is my name', additional_kwargs={}, example=False),
 AIMessage(content='Your name is Abhishek, based on our previous conversation. Is there anything else I can help you with?', additional_kwargs={}, example=False)]

## 5. LLAMA2 picks the tool (Kendra) and rephrased the question to search tool

In [363]:
agent_executor.run(input="What is the sponsorship agreement related to Kaya fest?")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e98b1970>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: sponsorship agreement Kaya fest

Observation:[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/qna-llm-data/legal/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformation Information S

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9920850>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Based on the search results, it appears that the sponsorship agreement related to Kaya Fest is a contract between Fruit of Life Productions LLC and Eco Science Solutions, Inc. The agreement outlines the terms and conditions of Eco Science Solutions' sponsorship of the Kaya Fest event, including the amount of $250,000.00 to be paid in full upon signing of the agreement, and various sponsorship benefits such as naming rights, vendor booths, VIP passes, and logo placement.

AI: The sponsorship agreement related to Kaya Fest appears to be a contract between Fruit of Life Productions LLC and Eco Science Solutions, Inc., outlining the terms and conditions of Eco Science Solutions' sponsorship of the event. Is there anything else I can help you with?

> Finished chain.


"The sponsorship agreement related to Kaya Fest appears to be a contract between Fruit of Life Productions LLC and Eco Science Solutions, Inc., outlining the terms and conditions of Eco Science Solutions' sponsorship of the event. Is there anything else I can help you with?"

## 6. LLAMA2 recalls from conversation history (Difficulty Level : Medium)

In [364]:
agent_executor.run(input="Who are the parties of this agreement and who is the sponsor?")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e99253a0>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: sponsorship agreement Kaya Fest parties

Observation:[Title: EcoScienceSolutions_Sponsorship_Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Sponsorship/EcoScienceSolutions_Sponsorship_Agreement.pdf, Passage content: SPONSORSHIP AGREEMENT This agreement (the "Agreement") is made effective this 1st day of April, 2018 ("Effective Date") between Fruit of Life Productions LLC, ("Promoter") and Eco Science Solutions, Inc.,("Sponsor"), 1135 Makawao Avenue, Suite 103-188, Makawao, Hawaii, 96768. Contribution by Sponsor: In consideration for the right to sponsor the Kaya Fest and to be acknowledged by Fruit of Life Productions LLC., as a Promoter of the event during the term of this Agreement, Sponsor agrees to contract with Fruit of Life Productions LLC., for the amount of $250,000.00 to be paid in full upon signing of this agreement. Bank Wire Transformation I

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e98b1850>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Based on the information provided, the parties involved in the sponsorship agreement for Kaya Fest are Fruit of Life Productions LLC (the promoter) and Eco Science Solutions, Inc. (the sponsor). The sponsor, Eco Science Solutions, has agreed to pay $250,000 to Fruit of Life Productions LLC in exchange for various sponsorship benefits, including the naming of the main stage after their brand, 4 10x10 on-site vendor booths, 50 VIP sponsor passes, and other perks.

> Finished chain.


' Based on the information provided, the parties involved in the sponsorship agreement for Kaya Fest are Fruit of Life Productions LLC (the promoter) and Eco Science Solutions, Inc. (the sponsor). The sponsor, Eco Science Solutions, has agreed to pay $250,000 to Fruit of Life Productions LLC in exchange for various sponsorship benefits, including the naming of the main stage after their brand, 4 10x10 on-site vendor booths, 50 VIP sponsor passes, and other perks.'

## 7. LLAMA2 rephrased the search term based on conversation memory and uses tool(Kendra) to search ( difficulty level : HIGH)

In [370]:
agent_executor.run(input="What is the address of this sponsor")

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9740220>: Failed to establish a new connection: [Errno 111] Connection refused'))




> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? YES
Action: Search
Action Input: Eco Science Solutions, Inc. address

Observation:[Title: EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement, URI: https://s3.us-east-1.amazonaws.com/anycorp-legaldata/Data/Endorsement/EcoScienceSolutionsInc_20171117_8-K_EX-10.1_10956472_EX-10.1_Endorsement Agreement.pdf, Passage content: 10. Notices. All notices provided for herein shall be given in writing by hand delivery, courier service, or by certified mail return receipt requested to the addresses of the parties set forth as follows (unless change of address by notice to the other party is given as provided in this paragraph 10): 3 Source: ECO SCIENCE SOLUTIONS, INC., 8-K, 11/17/2017 If to Talent: Mr. Stephen Marley 16115 SW 117t h Ave Suite 21-A Miami, Florida 33177 If to ESSI: Mr. Jeffery Taylor, President 1135 Makawao Avenue, Suite 103-188 Makawao, Hawaii 96768 with with a copy to: Sharon 

Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f21e9740700>: Failed to establish a new connection: [Errno 111] Connection refused'))


 Based on the information provided, the address of Eco Science Solutions, Inc. is not explicitly stated. However, the agreement mentions the addresses of the parties involved, which include:

* Mr. Stephen Marley: 16115 SW 117th Ave Suite 21-A Miami, Florida 33177
* Mr. Jeffery Taylor, President of ESSI: 1135 Makawao Avenue, Suite 103-188 Makawao, Hawaii 96768
* Sharon Mitchell, Corporate Counsel for ESSI: 829 Harcourt Rd. Grosse Pointe Park, Michigan 48230

Therefore, based on the information provided, the address of Eco Science Solutions, Inc. is not explicitly stated, but the addresses of the parties involved in the agreement are provided.

> Finished chain.


' Based on the information provided, the address of Eco Science Solutions, Inc. is not explicitly stated. However, the agreement mentions the addresses of the parties involved, which include:\n\n* Mr. Stephen Marley: 16115 SW 117th Ave Suite 21-A Miami, Florida 33177\n* Mr. Jeffery Taylor, President of ESSI: 1135 Makawao Avenue, Suite 103-188 Makawao, Hawaii 96768\n* Sharon Mitchell, Corporate Counsel for ESSI: 829 Harcourt Rd. Grosse Pointe Park, Michigan 48230\n\nTherefore, based on the information provided, the address of Eco Science Solutions, Inc. is not explicitly stated, but the addresses of the parties involved in the agreement are provided.'

In [366]:
memory.chat_memory.messages

[HumanMessage(content='my name is Abhishek', additional_kwargs={}, example=False),
 AIMessage(content="Hello Abhishek, it's nice to meet you. How can I assist you today?", additional_kwargs={}, example=False),
 HumanMessage(content='what is my name', additional_kwargs={}, example=False),
 AIMessage(content='Your name is Abhishek, based on our previous conversation. Is there anything else I can help you with?', additional_kwargs={}, example=False),
 HumanMessage(content='What is the sponsorship agreement related to Kaya fest?', additional_kwargs={}, example=False),
 AIMessage(content="The sponsorship agreement related to Kaya Fest appears to be a contract between Fruit of Life Productions LLC and Eco Science Solutions, Inc., outlining the terms and conditions of Eco Science Solutions' sponsorship of the event. Is there anything else I can help you with?", additional_kwargs={}, example=False),
 HumanMessage(content='Who are the parties of this agreement and who is the sponsor?', addition

In [270]:
memory.chat_memory.clear()
memory.chat_memory

ChatMessageHistory(messages=[])